### Accuracy Statistics

Using the validation_RF shapefile processed in QGIS

In [1]:
library(raster)
library(plyr)
setwd("~/RDemo/capstone/Sentinel2data")
img.classified <- raster("RF_classification2.tif")
shp.train <- shapefile("C://Users/Roxana/Documents/RDemo/capstone/QGISprocessed/training_data2.shp")
shp.valid <- shapefile("validation_RF")

Loading required package: sp


In [2]:
table(shp.valid$validclass)


 1  2  3  4  5  6  7  8 
 6 14  5 10 16  9  4  7 

We will generate two factor vectors, which we will compare in the confusion matrix:
1. **reference** : class labels assigned manually in QGIS
2. **predicted** : class labels that resulted in the automatic RF (or SVM) classification

In [4]:
reference <- as.factor(shp.valid$validclass)
reference # there are NA values, but we will ignore them later on

[1] 4    <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> 6    2    <NA> <NA> 2    <NA>
 [16] <NA> <NA> <NA> 5    <NA> <NA> <NA> <NA> 8    <NA> <NA> <NA> <NA> <NA> <NA>
 [31] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
 [46] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
 [61] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
 [76] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
 [91] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> 4    1    4    4    4    4   
[106] 5    2    3    2    5    <NA> <NA> 4    <NA> <NA> <NA> <NA> 7    <NA> 2   
[121] 4    <NA> 3    <NA> 5    <NA> 1    <NA> <NA> 2    1    <NA> <NA> 2    2   
[136] <NA> 6    <NA> 5    <NA> <NA> 1    5    4    5    <NA> <NA> 5    <NA> 5   
[151] <NA> <NA> 1    2    <NA> <NA> <NA> 5    7    2    <NA> 8    8    5    <NA>
[166] 1    6    <NA> 6    <NA> <NA> 5    <NA> <NA> 3    2    8    <NA> <NA> <NA>
[181] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> 5    <NA> 2    <NA> 7    <NA>
[196] <NA> 2    7    <NA> <NA> 8    <NA> 8    <NA> 5    6    <NA> <NA> <NA> <NA>
[211] 8    4    2    <NA> <NA> <NA> <NA> <NA> <NA> 3    <NA> 6    <NA> <NA> <NA>
[226] <NA> <NA> <NA> <NA> <NA> <NA> <NA> 3    5    <NA> <NA> <NA> <NA> <NA> <NA>
[241] <NA> <NA> <NA> <NA> <NA> <NA> 6    <NA> <NA> 6    <NA> <NA> <NA> <NA> <NA>
[256] <NA> <NA> <NA> <NA> <NA> <NA> <NA> 6    5    <NA> <NA> <NA> <NA> <NA> <NA>
[271] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
[286] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
[301] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
[316] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
[331] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
[346] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
[361] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
[376] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
[391] <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA>
Levels: 1 2 3 4 5 6 7 8

In [5]:
predicted <- as.factor(extract(img.classified, shp.valid))
predicted

[1] 4 2 4 7 8 2 1 2 5 6 6 8 3 2 4 3 3 7 5 5 4 5 4 8 7 8 4 2 8 2 2 8 7 8 5 5 2
 [38] 6 4 6 6 6 4 8 2 6 1 7 1 2 1 2 7 4 6 2 4 1 6 7 5 4 7 3 4 1 1 5 3 3 1 8 2 7
 [75] 1 1 8 8 2 8 5 1 5 5 7 6 8 5 2 1 3 4 4 1 8 5 6 7 7 4 7 4 3 4 4 5 2 3 6 5 5
[112] 5 3 6 4 8 8 8 1 2 7 3 1 5 5 7 1 4 3 2 1 7 4 2 2 7 6 4 1 5 5 1 1 4 5 4 2 1
[149] 2 5 1 4 1 2 8 5 5 1 6 2 8 8 8 1 5 7 6 5 6 4 8 5 2 8 3 2 8 7 3 1 8 7 5 7 3
[186] 8 4 7 1 5 5 2 1 7 3 5 2 6 1 3 8 6 8 4 5 8 6 3 4 1 8 4 6 4 7 3 2 1 3 3 3 6
[223] 6 3 2 7 1 2 6 2 1 8 8 5 1 4 7 2 3 8 3 5 4 8 5 7 5 1 1 7 4 3 2 8 3 7 1 6 4
[260] 4 4 2 8 5 7 4 4 2 7 8 7 4 4 3 6 3 6 5 2 3 1 2 8 7 8 7 2 8 6 5 6 8 4 8 7 1
[297] 3 3 3 1 6 6 3 5 3 3 2 3 5 2 6 1 3 6 2 6 3 6 8 6 7 6 8 7 1 7 1 3 7 8 6 1 3
[334] 2 6 8 2 3 4 6 7 7 7 1 6 1 4 5 3 2 7 8 5 6 1 8 4 7 6 5 8 6 2 8 6 2 4 6 4 3
[371] 3 7 3 7 6 7 3 1 4 2 1 7 3 6 1 2 2 7 5 4 5 3 5 5 5 7 6 3 4 6
Levels: 1 2 3 4 5 6 7 8

In [6]:
accmat <- table("pred" = predicted, "ref" = reference)
accmat # contingency table of the counts at each combination of factor levels

    ref
pred  1  2  3  4  5  6  7  8
   1  4  0  1  0  5  0  0  0
   2  0 11  0  0  0  0  0  0
   3  0  0  3  2  0  0  0  0
   4  0  0  0  7  0  0  0  0
   5  0  0  0  0 11  1  0  0
   6  0  3  0  0  0  5  2  0
   7  2  0  0  1  0  1  1  0
   8  0  0  1  0  0  2  1  7

The numbers reflect the number of validation pixels. All pixels that have a NA value in either reference or predicted were ignored here. This output already visualize if and where there are misclassifications in our map: all pixels located on the diagonale are correctly classified, all pixels off the diagonal are not.

**User’s accuracies:**

In [8]:
UA <- diag(accmat) / rowSums(accmat) * 100
round(UA, 2)

1      2      3      4      5      6      7      8 
 40.00 100.00  60.00 100.00  91.67  50.00  20.00  63.64

**Producer’s accuracies:**

In [10]:
PA <- diag(accmat) / colSums(accmat) * 100
round(PA,2) 

1      2      3      4      5      6      7      8 
 66.67  78.57  60.00  70.00  68.75  55.56  25.00 100.00

 **Overall accuracy:**

In [12]:
OA <- sum(diag(accmat)) / sum(accmat) * 100
round(OA,4)

[1] 69.0141

**Confusion matrix:**

In [13]:
accmat.ext <- addmargins(accmat)
accmat.ext <- rbind(accmat.ext, "Users" = c(PA, NA))
accmat.ext <- cbind(accmat.ext, "Producers" = c(UA, NA, OA))
colnames(accmat.ext) <- c(levels(as.factor(shp.train$classes)), "Sum", "PA")
rownames(accmat.ext) <- c(levels(as.factor(shp.train$classes)), "Sum", "UA")
accmat.ext <- round(accmat.ext, digits = 1)
dimnames(accmat.ext) <- list("Prediction" = colnames(accmat.ext),
                             "Reference" = rownames(accmat.ext))
class(accmat.ext) <- "table"
accmat.ext

          Reference
Prediction     b     c     f     g   ind    rh    rl     w   Sum    UA
       b     4.0   0.0   1.0   0.0   5.0   0.0   0.0   0.0  10.0  40.0
       c     0.0  11.0   0.0   0.0   0.0   0.0   0.0   0.0  11.0 100.0
       f     0.0   0.0   3.0   2.0   0.0   0.0   0.0   0.0   5.0  60.0
       g     0.0   0.0   0.0   7.0   0.0   0.0   0.0   0.0   7.0 100.0
       ind   0.0   0.0   0.0   0.0  11.0   1.0   0.0   0.0  12.0  91.7
       rh    0.0   3.0   0.0   0.0   0.0   5.0   2.0   0.0  10.0  50.0
       rl    2.0   0.0   0.0   1.0   0.0   1.0   1.0   0.0   5.0  20.0
       w     0.0   0.0   1.0   0.0   0.0   2.0   1.0   7.0  11.0  63.6
       Sum   6.0  14.0   5.0  10.0  16.0   9.0   4.0   7.0  71.0      
       PA   66.7  78.6  60.0  70.0  68.8  55.6  25.0 100.0        69.0

### Significance Test
We can also check if the result is purely coincidental, i.e., whether a random classification of the classes could have led to the same result, and will do this using a binomial test. Let x = total number of correctly classified validation points, and n = the total number of validation points in our confusion matrix:

In [17]:
sign <- binom.test(x = sum(diag(accmat)),
                   n = sum(accmat),
                   alternative = c("two.sided"),
                   conf.level = 0.95
)

**p-value:**

In [18]:
pvalue <- sign$p.value
pvalue

[1] 0.001820405

 **Confidence interval at alpha=0.05:**

In [16]:
conf_int_95 <- sign$conf.int[1:2]
round(conf_int_95,4)

[1] 0.5692 0.7946

### Conclusion: 
The *p-value* is lower than 0.05, so the classification resulted map is *somewhat significant*. If the classification were repeated under the same conditions, it can be assumed that the **overall accuracy** is 95% in the range of **56.92% to 79.46%**. 
Therefore, it is room for more model tuning, better resampling and more validation points, looking to further improve the model performance.